### Example notebook that:

### 1. Trains a CNN on laser-driven ion beam images labeled with their beam parameters
### 2. Trains a CNN to categorise each image based on how accurately the previous network was able to determine the beam parameters


### Part 1

In [ ]:
import matplotlib.pyplot as plt
import pickle
import utils
import numpy as np

from machine_learning.keras_trial import ml_trial
from analysis.analyser import Analyser, print_error_rates


%load_ext autoreload
%autoreload 2

In [ ]:
with open("../shared_data/50Kelectrons.pickle", "rb") as file:
    images_and_labels = pickle.load(file)


images = np.array(images_and_labels["images"])
labels = np.array(images_and_labels["labels"])[:, :5]

In [ ]:
index = 29019
print(labels[index])
plt.imshow(images[index])
plt.show()

Taking log of the number of protons and electrons to ensure a more linearly even distribution when normalised

In [ ]:
labels[:, 2] = np.log10(labels[:, 2])
labels[:, 4] = np.log10(labels[:, 4])

Normalising the Images and Labels


In [ ]:
images_scaler = utils.DynamicMinMaxScaler()
labels_scaler = utils.DynamicMinMaxScaler()
norm_images = images_scaler.fit_transform(images)
norm_labels = labels_scaler.fit_transform(labels)

Split into training and validation datsets

In [ ]:
FRAC_TRAIN = 3 / 4
n_train = int(FRAC_TRAIN * len(norm_images))


n_validate = len(norm_images) - n_train


x_train = norm_images[0:n_train]


y_train = norm_labels[0:n_train]


x_test = norm_images[n_train : n_train + n_validate]


y_test = norm_labels[n_train : n_train + n_validate]

Train the network with Keras

In [ ]:
BATCH_SIZE = 32
MAX_EPOCHS = 100
PATIENCE = 15
val_loss, model = ml_trial(
    x_train, y_train, x_test, y_test, BATCH_SIZE, MAX_EPOCHS, patience=PATIENCE
)

In [ ]:
norm_predictions = model.predict(x_test)

Restoring Original Label Sizes


In [ ]:
true_labels = labels_scaler.inverse_transform(y_test)
predictions = labels_scaler.inverse_transform(norm_predictions)
true_labels[:, 2] = 10 ** true_labels[:, 2]
predictions[:, 2] = 10 ** predictions[:, 2]
true_labels[:, 4] = 10 ** true_labels[:, 4]
predictions[:, 4] = 10 ** predictions[:, 4]

images_labels = {"images": x_test, "labels": true_labels, "predictions": predictions}

Printing the mean relative absolute errors for each parameter and the loss diagrams

In [ ]:
analyser = Analyser(
    images_labels,
    ["Proton E Max.", "Proton Temp.", "Proton N0", "Electron Temp.", "Electron N0"],
)


analyser.histogram_2d()


mraes = analyser.mraes()


print(mraes)

### Part 2

Categoring each image based on whether any of the parameters are above the relative absolute error threshold:

Proton Emax: 20%

Proton Temperature: 20%

Proton Number: 50%

Will be labelled 1 if any parameters exceed the thresholds above and 0 otherwise

In [ ]:
images_cats = analyser.categorise_by_threshold([0.2, 0.2, 0.5])
bin_images = images_cats["images"]
bin_labels = images_cats["labels"]

Checking the number of images labelled 1 and the number labelled 0

In [ ]:
n_positives = 0
for label in bin_labels:
    if label[0] == 1:
        n_positives += 1

print(f"No. 1s: {n_positives}")
print(f"No. 0s: {len(bin_labels) - n_positives}")

In [ ]:
FRAC_TRAIN = 3 / 4
n_train = int(FRAC_TRAIN * len(bin_images))


n_validate = len(bin_images) - n_train


x_train = bin_images[0:n_train]


y_train = bin_labels[0:n_train]


x_test = bin_images[n_train : n_train + n_validate]


y_test = bin_labels[n_train : n_train + n_validate]

Training a binary classifier CNN

In [ ]:
BATCH_SIZE = 32
MAX_EPOCHS = 10
val_loss, categoriser = ml_trial(
    x_train,
    y_train,
    x_test,
    y_test,
    BATCH_SIZE,
    MAX_EPOCHS,
    patience=15,
    loss="binary_crossentropy",
    out_activation="sigmoid",
)

Collapsing the predictions to 0 or 1 based on a user specified cutoff

1 if prediction >= CUTOFF

0 if prediction < CUTOFF

In [ ]:
CUTOFF = 0.5
predictions = categoriser.predict(x_test)
predictions = np.array([1 if prediction >= CUTOFF else 0 for prediction in predictions])
predictions = predictions.astype(int)

In [ ]:
print_error_rates(y_test, predictions)